## Imports

In [3]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

In [4]:
metadata = pd.read_csv('../src/meta_data.csv')

# Define the image size
IMG_HEIGHT = 128
IMG_WIDTH = 128

def load_images(image_folder, metadata):
    images = []
    labels = []
    for index, row in tqdm(metadata.iterrows(), desc="Processing", total=len(metadata)):
        img_path = os.path.join(image_folder, row['Filename'])
        if os.path.exists(img_path):
            img = load_img(img_path, target_size=(IMG_HEIGHT, IMG_WIDTH))
            img_array = img_to_array(img)
            images.append(img_array)
            labels.append(row['Model'])
    return np.array(images), np.array(labels)

# Load images and labels
image_folder = '../src/extracted_data'
images, labels = load_images(image_folder, metadata)

# Normalize images
images = images / 255.0

# Encode labels
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
labels_categorical = to_categorical(labels_encoded)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels_categorical, test_size=0.2, random_state=42)

Processing: 100%|██████████| 20102/20102 [00:24<00:00, 804.90it/s]


In [5]:
def create_cnn_model(input_shape, num_classes):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    return model

input_shape = (IMG_HEIGHT, IMG_WIDTH, 3)
num_classes = len(label_encoder.classes_)
model = create_cnn_model(input_shape, num_classes)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [6]:
# Train the model
history = model.fit(X_train, y_train, epochs=20, validation_split=0.2, batch_size=32)

# Save the model
model.save('../src/models')

Epoch 1/20
189/402 [=============>................] - ETA: 58s - loss: 1.5946 - accuracy: 0.3489

In [ ]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_accuracy}')